In [6]:
import pandas as pd
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer

# TF-IDF Weight

In [7]:
hc_files = Path('./data/hra/control').glob('*.csv')
ad_files = Path('./data/hra/dementia').glob('*.csv')

lower_list = [chr(x) for x in range(97, 123)]
digit2id_list = list(range(10,33))

hc = []
for f in hc_files:
    df = pd.read_csv(f)
    s = df['x']
    
    for i in range(len(digit2id_list)):
        s = s.replace(digit2id_list[i], lower_list[i])
        
    hc.append(''.join([str(c) for c in s.to_list()]))
    
ad = []
for f in ad_files:
    df = pd.read_csv(f)
    s = df['x']
    
    for i in range(len(digit2id_list)):
        s = s.replace(digit2id_list[i], lower_list[i])
        
    ad.append(''.join([str(i) for i in s.to_list()]))

In [8]:
char_vectorizer_hc = TfidfVectorizer(
    analyzer='char',
    ngram_range=(1,1),
    sublinear_tf=True)

char_vectorizer_ad = TfidfVectorizer(
    analyzer='char',
    ngram_range=(1,1),
    sublinear_tf=True)

tfidf_hc = char_vectorizer_hc.fit_transform(hc)
tfidf_ad = char_vectorizer_ad.fit_transform(ad)

tfidf_mx_hc = pd.DataFrame(data=tfidf_hc.toarray(), columns=char_vectorizer_hc.get_feature_names())
tfidf_mx_ad = pd.DataFrame(data=tfidf_ad.toarray(), columns=char_vectorizer_ad.get_feature_names())
label1 = [0 for _ in range(tfidf_mx_hc.shape[0])]
label2 = [1 for _ in range(tfidf_mx_ad.shape[0])]

In [26]:
rank_hc = tfidf_mx_hc.rank(axis=1,method='first',ascending=False)
rank_hc

,0,1,2,3,4,5,6,7,8,9,...,m,n,o,p,q,r,s,t,u,v
0,1.0,29.0,30.0,21.0,13.0,31.0,11.0,22.0,18.0,15.0,...,32.0,9.0,5.0,4.0,16.0,27.0,20.0,19.0,23.0,2.0
1,1.0,27.0,28.0,18.0,11.0,29.0,10.0,19.0,15.0,12.0,...,31.0,9.0,6.0,4.0,20.0,26.0,22.0,14.0,23.0,32.0
2,1.0,21.0,23.0,14.0,12.0,28.0,9.0,24.0,18.0,13.0,...,29.0,11.0,6.0,3.0,19.0,30.0,22.0,31.0,20.0,32.0
3,1.0,12.0,29.0,28.0,15.0,25.0,7.0,26.0,18.0,14.0,...,30.0,11.0,9.0,2.0,16.0,24.0,20.0,31.0,17.0,32.0
4,1.0,27.0,29.0,26.0,10.0,18.0,3.0,21.0,15.0,13.0,...,30.0,9.0,7.0,2.0,17.0,24.0,20.0,31.0,22.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
238,1.0,31.0,13.0,24.0,19.0,28.0,9.0,22.0,20.0,12.0,...,29.0,10.0,8.0,3.0,16.0,27.0,18.0,23.0,26.0,32.0
239,1.0,23.0,30.0,22.0,12.0,24.0,8.0,25.0,15.0,16.0,...,31.0,11.0,10.0,6.0,21.0,29.0,20.0,32.0,26.0,2.0
240,1.0,30.0,31.0,19.0,13.0,25.0,9.0,27.0,20.0,15.0,...,26.0,10.0,6.0,4.0,16.0,28.0,22.0,18.0,23.0,32.0
241,1.0,29.0,28.0,22.0,13.0,30.0,9.0,24.0,18.0,11.0,...,27.0,10.0,8.0,5.0,16.0,26.0,20.0,19.0,25.0,32.0


In [27]:
rank_ad = tfidf_mx_ad.rank(axis=1,method='first',ascending=False)
rank_ad

,0,1,2,3,4,5,6,7,8,9,...,m,n,o,p,q,r,s,t,u,v
0,1.0,27.0,28.0,19.0,13.0,29.0,5.0,21.0,15.0,11.0,...,30.0,10.0,8.0,3.0,16.0,20.0,17.0,31.0,23.0,32.0
1,1.0,28.0,20.0,14.0,11.0,29.0,5.0,23.0,21.0,12.0,...,30.0,10.0,9.0,3.0,22.0,26.0,19.0,31.0,16.0,32.0
2,1.0,23.0,29.0,14.0,15.0,18.0,6.0,24.0,19.0,12.0,...,30.0,10.0,8.0,3.0,20.0,26.0,13.0,31.0,27.0,32.0
3,1.0,16.0,28.0,12.0,11.0,29.0,7.0,19.0,21.0,22.0,...,30.0,6.0,10.0,4.0,17.0,25.0,27.0,31.0,23.0,32.0
4,1.0,19.0,20.0,16.0,12.0,21.0,9.0,15.0,22.0,23.0,...,28.0,14.0,11.0,3.0,29.0,30.0,18.0,31.0,17.0,32.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,1.0,27.0,29.0,17.0,12.0,24.0,10.0,22.0,19.0,14.0,...,30.0,11.0,7.0,3.0,18.0,28.0,16.0,31.0,26.0,32.0
305,1.0,26.0,29.0,17.0,14.0,30.0,9.0,22.0,21.0,10.0,...,25.0,12.0,8.0,2.0,15.0,27.0,20.0,31.0,23.0,32.0
306,1.0,28.0,29.0,18.0,16.0,12.0,9.0,21.0,20.0,10.0,...,30.0,11.0,7.0,2.0,15.0,25.0,23.0,31.0,24.0,32.0
307,1.0,26.0,27.0,12.0,10.0,28.0,6.0,24.0,16.0,18.0,...,30.0,11.0,7.0,2.0,21.0,20.0,25.0,31.0,22.0,32.0


# Encoding the data according to the TF-IDF weight matrix

In [35]:
hc[0]

'pda0o8aja0eo0ej0pda0gep8daj040pda0ikpdan0eo0selejc09eodao06j90pda0s6pan0eo0nqjjejc0kj0pda0bhkkn040607ku0eo0pnuejc0pk0cap08kkgeao0kqp0kb060f6n06j90da3o067kqp0pk0pel0kran0kj060opkkh040pda0heppha0cenh0eo0na68pejc0pk0deo0b6hhejc040ep0oaaio0pk07a0oqiian0kqp040pda0sej9ks0eo0klaj040pda08qnp6ejo06na07hksejc040ep0iqop07a060cajpha07naava040pdana3o0cn6oo0kqpoe9a0ej0pda0c6n9aj040ikpdan3o0bejeoda908anp6ej0kb0pda09eodao040gep8daj3o0ranu0pe9u040pda0ikpdan0oaaio0pk0d6ra0jkpdejc0ej0pda0dkqoa0pk0a6p0at8alp08kkgeao0ej0pda08kkgea0f6n040pda08deh9naj0hkkg0pk07a06hikop067kqp0pda0o6ia0oeva040land6lo0pdau3na0psejo040pdau3na09naooa90bkn0oqiian0s6ni0sa6pdan040ukq0s6jp0ikna010pda0ikpdan3o0ej060odknp0ohaara09naoo040e3hh0d6bp60o6u0ep3o0s6ni04'

In [54]:
temp_code_hc = rank_hc.columns.to_list()
for n,s in enumerate(hc[0:1]):
    tfidf_s = []
    for c in s:
        tfidf_s.append(rank_hc.iloc[n,temp_code_hc.index(c)])
        tfidf_sdf = pd.DataFrame({'x':tfidf_s})
        tfidf_sdf.index.name = 'index'
        # tfidf_sdf.to_csb(Path.cwd()/'data'/'hra'/'tf-idf_control'/(str(n)+'.csv'))
tfidf_sdf

,x
index,
0,4.0
1,8.0
2,3.0
3,1.0
4,5.0
...,...
717,11.0
718,9.0
719,12.0


In [57]:
temp_code_hc = rank_hc.columns.to_list()
for n,s in enumerate(hc):
    tfidf_s = []
    for c in s:
        tfidf_s.append(rank_hc.iloc[n,temp_code_hc.index(c)])
        tfidf_sdf = pd.DataFrame({'x':tfidf_s})
        tfidf_sdf.index.name = 'index'
        tfidf_sdf.to_csv(Path.cwd()/'data'/'hra'/'tf-idf_control'/(str(n)+'.csv'))
        
temp_code_ad = rank_ad.columns.to_list()
for n,s in enumerate(ad):
    tfidf_s = []
    for c in s:
        tfidf_s.append(rank_ad.iloc[n,temp_code_ad.index(c)])
        tfidf_sdf = pd.DataFrame({'x':tfidf_s})
        tfidf_sdf.index.name = 'index'
        tfidf_sdf.to_csv(Path.cwd()/'data'/'hra'/'tf-idf_dementia'/(str(n)+'.csv'))